In [4]:
import pandas as pd                    # data package
import matplotlib.pyplot as plt        # graphics module  
import datetime as dt                  # date and time module
import numpy as np                     # foundation for pandas 

In [11]:
data_file = "./data/WIOT2014_Nov16_ROW.xlsb" 
# Just download as is from WIOT and the hope is to have the file setup so things are nice and clean

df = pd.read_excel(data_file, engine='pyxlsb')

traded = {"r1", "r2", "r3", "r4", "r5", "r6", "r7", "r8", "r9", "r10","r11","r12","r13","r14","r15","r16","r17","r18","r19","r20","r21","r22"}

total = {"r65"}

country_list = { 'AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'CHN', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 
               'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'MEX', 'NLD', 'NOR', 'PRT', 'SWE', 'TWN', 'USA'}

In [12]:
df.head()

,Intercountry Input-Output Table,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 2679,Unnamed: 2680,Unnamed: 2681,Unnamed: 2682,Unnamed: 2683,Unnamed: 2684,Unnamed: 2685,Unnamed: 2686,Unnamed: 2687,Unnamed: 2688
0,"43 countries, in current prices",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(industry-by-industry),NaN,NaN,NaN,A01,A02,A03,B,C10-C12,C13-C15,...,CONS_np,CONS_g,GFCF,INVEN,CONS_h,CONS_np,CONS_g,GFCF,INVEN,GO
2,(millions of US$),NaN,NaN,NaN,"Crop and animal production, hunting and relate...",Forestry and logging,Fishing and aquaculture,Mining and quarrying,"Manufacture of food products, beverages and to...","Manufacture of textiles, wearing apparel and l...",...,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Final consumption expenditure by households,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Total output
3,NaN,NaN,NaN,NaN,AUS,AUS,AUS,AUS,AUS,AUS,...,USA,USA,USA,USA,ROW,ROW,ROW,ROW,ROW,TOT
4,NaN,NaN,NaN,NaN,c1,c2,c3,c4,c5,c6,...,c58,c59,c60,c61,c57,c58,c59,c60,c61,c62


In [62]:
bar = df[df.iloc[:,3].isin(traded)]

In [63]:
bar

,Intercountry Input-Output Table,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 2679,Unnamed: 2680,Unnamed: 2681,Unnamed: 2682,Unnamed: 2683,Unnamed: 2684,Unnamed: 2685,Unnamed: 2686,Unnamed: 2687,Unnamed: 2688
5,A01,"Crop and animal production, hunting and relate...",AUS,r1,12924.179691,112.454221,228.480462,538.890807,25139.334287,737.492591,...,0,0.000036,0.324914,0.027391,723.189984,0.000374,14.15635,20.520708,0.010083,70292.034492
6,A02,Forestry and logging,AUS,r2,83.029637,201.499948,0.191313,17.190679,2.646331,0.093768,...,0,0.000002,0.015015,0.000439,0.423422,0.000003,0.000071,0.000977,-0.000109,2585.379685
7,A03,Fishing and aquaculture,AUS,r3,19.147731,0.001079,19.106701,4.228108,316.075149,22.866683,...,0,0.000001,0.004963,0.008311,404.041139,0.000386,0.004619,0.357978,0.000592,3175.044396
8,B,Mining and quarrying,AUS,r4,115.929847,0.69504,5.924477,4492.037652,331.609356,5.984141,...,0,0.000177,1.610294,0.047027,14.871627,0.006665,0.594705,19.726833,-0.282072,171985.122327
9,C10-C12,"Manufacture of food products, beverages and to...",AUS,r5,1590.84059,0.880063,42.54076,203.011938,10851.759013,262.228559,...,0,0.000153,1.393463,0.041279,4810.234988,0.001371,47.016724,1.345248,0.027914,83504.036584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,C28,Manufacture of machinery and equipment n.e.c.,ROW,r19,37.341429,0.086027,9.761812,289.971553,27.806322,0.712649,...,0,0.000001,2095.697847,42.977449,1517.011544,0.047876,40.708612,11898.875972,66444.722402,493037.162264
2432,C29,"Manufacture of motor vehicles, trailers and se...",ROW,r20,5.205023,0.004969,1.805816,25.179996,2.00901,0.314073,...,0,0.000014,1495.783817,29.851333,112087.504387,0.086908,2055.669509,54661.493073,45005.658511,601211.269047
2433,C30,Manufacture of other transport equipment,ROW,r21,4.766235,0.001476,6.472586,30.474833,0.759728,0.105959,...,0,0.000001,392.531815,61.941502,0,0,0,0,41849.281263,160783.608461
2434,C31_C32,Manufacture of furniture; other manufacturing,ROW,r22,1.563513,0.004699,0.229549,4.326001,1.697991,0.706743,...,0,0.030135,2601.677481,95.227401,0,0,0,0,4101.235472,143166.025235


In [96]:
condition = ( df.iloc[4,:] == "c57") & (df.iloc[3,:] == "USA")

row_condition = ( df.iloc[:,3].isin(traded) ) & (df.iloc[:,2].isin(country_list))

foo = df.loc[row_condition,condition]

In [97]:
foo.sum()

Unnamed: 2678    380353.659075
dtype: object

In [88]:
foo.sum()

Unnamed: 2678    1948461.789907
dtype: object

In [98]:
380353 / 1948461

0.1952068837918747

In [40]:
df.iloc[4,:] =="c1"

Intercountry Input-Output Table    False
Unnamed: 1                         False
Unnamed: 2                         False
Unnamed: 3                         False
Unnamed: 4                          True
                                   ...  
Unnamed: 2684                      False
Unnamed: 2685                      False
Unnamed: 2686                      False
Unnamed: 2687                      False
Unnamed: 2688                      False
Name: 4, Length: 2689, dtype: bool

In [82]:
traded = {"r1", "r2", "r3", "r4", "r5", "r6", "r7", "r8", "r9", "r10","r11","r12","r13","r14","r15","r16","r17","r18","r19","r20","r21","r22"}

In [51]:
df.iloc[:,4].isin(traded)

0       False
1       False
2       False
3       False
4       False
        ...  
2472    False
2473    False
2474    False
2475    False
2476    False
Name: Unnamed: 4, Length: 2477, dtype: bool

In [58]:
df.iloc[:,3]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
2472    r68
2473    r69
2474    r70
2475    r71
2476    r73
Name: Unnamed: 3, Length: 2477, dtype: object

In [95]:
country_list = { 'AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'CHN', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 
               'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'MEX', 'NLD', 'NOR', 'PRT', 'SWE', 'TWN', 'USA'}

In [86]:
all_country_list = {'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP',
       'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC',
       'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU',
       'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU',
       'RUS', 'SVK', 'SVN', 'SWE', 'TUR', 'TWN', 'USA', 'ROW'}